Reference:

[**Introduction to LangChain for Agentic AI**](https://courses.analyticsvidhya.com/courses/take/introduction-to-langchain-for-agentic-ai/lessons/61748853-course-introduction) 

## Install OpenAI, and LangChain dependencies

In [1]:
# !pip install -qq langchain
# !pip install -qq langchain-openai
# !pip install -qq langchain-community
# !pip install -qq langchain-chroma

## Enter Open AI API Key

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## Load Connection to LLM

Here we create a connection to ChatGPT to use later in our chains

In [2]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

## Multi-user Window-based Conversation Chains with persistence - SQLChatMessageHistory

The beauty of `SQLChatMessageHistory` is that we can store separate conversation histories per user or session which is often the need for real-world chatbots which will be accessed by many users at the same time. Instead of in-memory we can store it in a SQL database which can be used to store a lot of conversations.

We use a `get_session_history` function which is expected to take in a `session_id` and return a Message History object. Everything is stored in a SQL database. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain

We also use a `memory_buffer_window` function to only use the top-K last historical conversations before sending it to the LLM, basically our own implementation of `ConversationBufferWindowMemory`


- `ChatMessageHistory`: This is a class in LangChain that stores a sequence of chat messages (such as HumanMessage, AIMessage, SystemMessage) for a conversation. It allows you to append new messages and retrieve the full message history, which is useful for maintaining conversational context.

- `BaseChatMessageHistory`: This is an abstract base class that defines the interface for chat message history storage backends. It specifies methods like add_message, get_messages, and clear, which concrete implementations (like ChatMessageHistory, RedisChatMessageHistory, etc.) must provide.

- `RunnableWithMessageHistory`: This is a wrapper for a Runnable (such as a chain or LLM) that automatically manages message history for each session or user. It uses a function (like get_session_history) to retrieve the appropriate message history object based on a session ID, and injects the history into the chain's input. This enables multi-user or multi-session conversational experiences.

- `MessagesPlaceholder`: This is a special prompt template variable in LangChain that acts as a placeholder for a list of messages (e.g., the conversation history). When rendering a prompt, MessagesPlaceholder is replaced with the actual messages from history, allowing the LLM to see the full conversation context.


In [3]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove all conversation histories
# !rm memory.db

In [19]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# prompt to load in history and current input from the user
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{human_input}"),
    ]
)

# create a memory buffer window function to return the last K conversations
def memory_buffer_window(messages, k=2):
    return messages[-(k+1):]

# create a basic LLM Chain which only sends the last K conversations per user
llm_chain = (
    RunnablePassthrough.assign(history=lambda x: memory_buffer_window(x["history"]))
      |
    prompt_template
      |
    chatgpt
      | 
    StrOutputParser()
)

In [22]:
# create a conversation chain which can load memory based on specific user or session id
conv_chain = RunnableWithMessageHistory(
    llm_chain,
    get_session_history_db,
    input_messages_key="human_input",
    history_messages_key="history",
)

# create a utility function to take in current user input prompt and their session ID
# streams result live back to the user from the LLM
def chat_with_llm(prompt: str, session_id: str):
    for chunk in conv_chain.stream({"human_input": prompt},
                                   {'configurable': { 'session_id': session_id}}):
        # print(chunk.content, end="")
        print(chunk, end="")

In [23]:
user_id = 'jim001'
prompt = "Hi can you tell me which is the fastest animal?"
chat_with_llm(prompt, user_id)

The fastest animal in the world is the peregrine falcon. When in a dive, it can reach speeds of over 240 miles per hour (386 kilometers per hour). If you're considering speed in level flight, the Brazilian free-tailed bat holds the record, flying at speeds of around 99 miles per hour (160 kilometers per hour). For land animals, the cheetah is the fastest, capable of running speeds up to 60-70 miles per hour (97-113 kilometers per hour) in short bursts.

In [24]:
prompt = "what about the slowest animal?"
chat_with_llm(prompt, user_id)

The slowest animal is often considered to be the three-toed sloth. It moves at an average speed of about 0.24 kilometers per hour (0.15 miles per hour) when climbing trees. In water, they can swim slightly faster, but they are still quite slow compared to many other animals. Another contender for the title of slowest animal is the garden snail, which moves at a speed of about 0.03 miles per hour (0.048 kilometers per hour). Both of these animals are well-adapted to their environments, despite their slow pace.


#### ![Chat Message Memory Flow](../images/sqllite.png)

In [25]:
prompt = "what about the largest animal?"
chat_with_llm(prompt, user_id)

The largest animal on Earth is the blue whale (*Balaenoptera musculus*). Blue whales can reach lengths of up to 100 feet (30 meters) and can weigh as much as 200 tons (approximately 181 metric tonnes) or more. They are not only the largest animals alive today but also the largest animals known to have ever existed on our planet. Their immense size is complemented by their unique feeding habits, as they primarily consume tiny shrimp-like animals called krill.

In Below cell we will see the LLM always will get last 2 message in its history as we have kept window = 2

In [26]:
prompt = "what topics have we discussed, show briefly as bullet points"
chat_with_llm(prompt, user_id)

Sure! Here are the topics we've discussed so far:

- The slowest animal (three-toed sloth and garden snail)
- The largest animal (blue whale)

In [27]:
user_id = 'john005'
prompt = "Explain AI in 3 bullets to a child"
chat_with_llm(prompt, user_id)

Sure! Here are three simple points to explain AI to a child:

1. **Smart Helpers**: AI is like a smart robot or computer that can help us do things, like answering questions or playing games, just like a friend would.

2. **Learning from Experience**: AI learns from lots of information, kind of like how you learn from your school lessons or by practicing. The more it learns, the better it gets at helping us!

3. **Talking and Understanding**: AI can understand what we say or write and can even talk back to us, making it feel like we’re having a conversation with a really smart buddy!

In [28]:
prompt = "Now do the same for Generative AI"
chat_with_llm(prompt, user_id)

Of course! Here are three simple points to explain Generative AI to a child:

1. **Creative Robot**: Generative AI is like a creative robot that can make new things, like stories, pictures, or music, all by itself, just like an artist or a writer!

2. **Using Ideas**: It learns from lots of examples, like reading many books or looking at many pictures, so it can come up with its own cool ideas based on what it has learned.

3. **Imagination in Action**: When you ask Generative AI to create something, it uses its imagination to come up with fun and interesting things, almost like magic!

In [29]:
prompt = "Now do the same for machine learning"
chat_with_llm(prompt, user_id)

Sure! Here are three simple points to explain machine learning to a child:

1. **Learning Like You**: Machine learning is when computers learn from examples, just like you learn new things in school by practicing and seeing how things work.

2. **Getting Better Over Time**: The more examples the computer sees, the better it gets at understanding and making decisions, like how you get better at a game the more you play it.

3. **Solving Problems**: Machine learning helps computers solve problems or make predictions, like guessing what movie you might like based on the ones you’ve watched before, just like how your friends might know your favorite games!

In [30]:
prompt = "what topics have we discussed, show briefly as bullet points"
chat_with_llm(prompt, user_id)

Sure! Here are the topics we've discussed so far:

- **Generative AI**
  - Creative robot that makes new things (stories, pictures, music)
  - Learns from examples to come up with ideas
  - Uses imagination to create fun and interesting content

- **Machine Learning**
  - Computers learn from examples like humans do
  - Improves over time with practice and experience
  - Helps solve problems and make predictions based on data

If you have more questions or need further information on these topics, feel free to ask!

## Alternative Implementation: RunnableWithMessageHistory with Full History

This is an alternative implementation demonstrating `RunnableWithMessageHistory` with:
1. Full conversation history (no windowing)
2. Custom session history factory
3. Cleaner separation of concerns


In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

# Create prompt template with history placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Be concise and informative."),
    MessagesPlaceholder(variable_name="history"),  # This will be populated with chat history
    ("human", "{input}")
])

# Create the base chain (prompt | llm)
base_chain = prompt | llm

# Session history factory function - creates/retrieves history for each session
def get_session_history(session_id: str) -> SQLChatMessageHistory:
    """
    Factory function that returns a SQLChatMessageHistory instance for the given session.
    Each session_id gets its own conversation history stored in SQLite.
    """
    return SQLChatMessageHistory(
        session_id=session_id,
        connection="sqlite:///chat_history_alt.db"  # Using a different DB file
    )

# Wrap the chain with message history management
chain_with_history = RunnableWithMessageHistory(
    base_chain,
    get_session_history,
    input_messages_key="input",      # Key in input dict for the current message
    history_messages_key="history"   # Key in prompt for the message history
)

# Helper function to chat with a specific user session
def chat(user_input: str, session_id: str) -> str:
    """Send a message and get a response, maintaining conversation history per session."""
    response = chain_with_history.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": session_id}}
    )
    return response.content

# Streaming version of the chat function
def chat_stream(user_input: str, session_id: str):
    """Send a message and stream the response."""
    for chunk in chain_with_history.stream(
        {"input": user_input},
        config={"configurable": {"session_id": session_id}}
    ):
        print(chunk.content, end="", flush=True)
    print()  # New line at the end


### Test with User 1 (Alice)


In [ ]:
# User 1: Alice asks about Python
print("Alice:", chat("My name is Alice. What are the best practices for Python programming?", "alice_session"))


### Test with User 2 (Bob) - Different Session


In [ ]:
# User 2: Bob asks about JavaScript (separate conversation history)
print("Bob:", chat("My name is Bob. What's new in JavaScript ES2024?", "bob_session"))


### Verify Memory Isolation - Each user remembers their own context


In [ ]:
# Alice asks a follow-up - should remember her name and previous topic (Python)
print("Alice follow-up:", chat("What's my name? And what were we discussing?", "alice_session"))


In [ ]:
# Bob asks a follow-up - should remember his name and previous topic (JavaScript)
print("Bob follow-up:", chat("What's my name? And what were we discussing?", "bob_session"))


### View Stored Conversation History


In [ ]:
# View Alice's conversation history stored in the database
alice_history = get_session_history("alice_session")
print("=== Alice's Conversation History ===")
for msg in alice_history.messages:
    role = "Human" if msg.type == "human" else "AI"
    print(f"{role}: {msg.content[:100]}...")

In [ ]:
# View Bob's conversation history stored in the database
bob_history = get_session_history("bob_session")
print("=== Bob's Conversation History ===")
for msg in bob_history.messages:
    role = "Human" if msg.type == "human" else "AI"
    print(f"{role}: {msg.content[:100]}...")

In [ ]:
prompt = "what topics have we discussed, show briefly as bullet points"
chat_with_llm(prompt, user_id)

Sure! Here are the topics we've discussed so far:

- The slowest animal (three-toed sloth and garden snail)
- The largest animal (blue whale)

Sure! Here are three simple points to explain AI to a child:

1. **Smart Helpers**: AI is like a smart robot or computer that can help us do things, like answering questions or playing games, by learning from lots of information.

2. **Learning from Experience**: Just like you learn new things by practicing, AI learns by looking at examples and figuring out patterns, so it can get better over time.

3. **Talking and Understanding**: AI can understand what we say and even talk back to us, like a friend who knows a lot and can help us find answers or tell stories!

Of course! Here are three simple points to explain Generative AI to a child:

1. **Creative Robot**: Generative AI is like a creative robot that can make new things, like pictures, stories, or music, all by itself!

2. **Learning from Examples**: It learns by looking at lots of examples, just like how you get ideas for your drawings or stories by seeing what others have made.

3. **Imagination in Action**: When you ask Generative AI to create something, it uses its imagination to come up with fun and unique ideas, almost like magic!

Sure! Here are three simple points to explain machine learning to a child:

1. **Learning Like You**: Machine learning is when computers learn from examples, just like you learn new things by practicing and trying different activities.

2. **Finding Patterns**: It helps computers find patterns in information, like how you notice that some animals have stripes or spots, so they can make smart guesses about new things.

3. **Getting Better Over Time**: The more examples the computer sees, the better it gets at understanding and making decisions, kind of like how you get better at a game the more you play it!

Sure! Here are the topics we've discussed so far:

- **Generative AI**
  - Creative robot that makes new things
  - Learns from examples
  - Uses imagination to create unique ideas

- **Machine Learning**
  - Computers learn like humans do
  - Finds patterns in information
  - Improves over time with more examples